In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
from pathlib import Path

sys.path.append(os.path.join(Path().resolve(), '../../../../..'))

import numpy as np
from tqdm import tqdm

import hydra
from omegaconf import DictConfig, OmegaConf
import torch


/home/docker/.pyenv/versions/3.8.0/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 勾配を計算しない
torch.set_grad_enabled(False)

In [3]:
%matplotlib inline
# %matplotlib notebook
# %matplotlib nbagg
from IPython import display
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, ArtistAnimation

import numpy as np
from tqdm import tqdm

In [4]:
model_folder = "results/test2-seed_0/2022-12-07/run_0"

In [7]:
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf
with initialize(config_path=model_folder):
    cfg = compose(config_name="hydra_config")

cfg_device = "cuda:0"
# cfg_device = "cpu"
cfg.main.device = cfg_device
# cfg.train.n_episode = 100
print(' ' * 26 + 'Options')
for k, v in cfg.items():
    print(' ' * 26 + k + ': ' + str(v))

device = torch.device(cfg.main.device)

                          Options
                          main: {'experiment_name': 'test2-seed_0', 'tags': ['RSSM'], 'log_dir': '/home/docker/sharespace/Multimodal-RSSM/train/HF-PGM/House/MRSSM/MRSSM/results/test2-seed_0/2022-12-07/run_0', 'seed': 0, 'disable_cuda': False, 'device': 'cuda:0', 'wandb': True, 'git_hash': '80dccb5'}
                          env: {'env_config': {'env_name': 'House'}, 'taskset_name': 'HF-PGM', 'observation_shapes': {'image_hsr_256': [3, 256, 256]}, 'action_name': 'Pose', 'action_size': 7, 'action_repeat': 1, 'bit_depth': 5, 'max_episode_length': None}
                          rssm: {'observation_names_enc': ['image_hsr_256'], 'observation_names_rec': ['image_hsr_256'], 'condition_names': ['Pose'], 'predict_reward': False, 'multimodal': True, 'multimodal_params': {'fusion_timing': 'obs_emb', 'fusion_method': 'PoE', 'num_components': None}, 'MuMMI': {'use': False}, 'dreaming': {'use': False, 'cpc_overshooting_distance': 3, 'cpc_contrast': 'window', 'mycp

/tmp/ipykernel_134094/3059289892.py:3: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with initialize(config_path=model_folder):


In [8]:
# cfg.emv.action_name = "action"
# cfg.rssm.condition_names = ["action"]

# Load Model, Data and States

In [9]:
import glob
model_paths = glob.glob(os.path.join(model_folder, '*.pth'))
print("model_pathes: ")
model_paths

model_pathes: 


['results/test2-seed_0/2022-12-07/run_0/models_1000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_2000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_3000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_4000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_5000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_6000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_7000.pth',
 'results/test2-seed_0/2022-12-07/run_0/models_8000.pth']

In [10]:
model_idx = 3

from algos.MRSSM.MRSSM.algo import build_RSSM

model = build_RSSM(cfg, device)
model_path = model_paths[model_idx]
model.load_model(model_path)
model.eval()



Multimodal RSSM (fusion obs_emb by PoE)
load model_dicts from results/test2-seed_0/2022-12-07/run_0/models_4000.pth


/home/docker/.pyenv/versions/3.8.0/lib/python3.8/site-packages/torch/optim/adam.py:90: UserWarning: optimizer contains a parameter group with duplicate parameters; in future, this will cause an error; see github.com/pytorch/pytorch/issues/40967 for more information
  super(Adam, self).__init__(params, defaults)


In [11]:
use_validation_data = True

In [12]:
from algos.MRSSM.MRSSM.train import get_dataset_loader

cwd = "."
if use_validation_data:
    D = get_dataset_loader(cfg, cwd, device, cfg.train.validation_data_path)
else:
    D = get_dataset_loader(cfg, cwd, device, cfg.train.train_data_path)


load dataset from ./../../../../../dataset/HF-PGM/MobileRobot_with_Image_Pose/MobileRobotImage_20221127/validation
find 3 npy files!


load dataset: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  3.35it/s]

set color augment params
calc pca from torch.Size([87, 3, 258, 258]) data


# Reconstruction

In [13]:
from utils.evaluation.estimate_states import get_episode_data

epi_idx = 0
crop_idx = 0
observations, actions, rewards, nonterminals = get_episode_data(D, epi_idx=epi_idx, crop_idx=crop_idx)

In [14]:
observations.keys()

dict_keys(['image_hsr_256'])

In [15]:
size = (256,256)
size = (64,64)

In [16]:
!pip install scikit-learn


[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


### get PCA result

In [17]:
from utils.evaluation.visualize_utils import get_pca_model, tensor2np, get_xyz

#### <span style="color: red; ">please run estimate_state.py for estimate states of train dataset</span>

In [18]:
# # load states
# state_path = model_path.replace("models", "states_models").replace(".pth", ".npy")
# print("state_path:", state_path)

# states_np = np.load(state_path, allow_pickle=True).item()
# # print("-- dataset --")
# # for key in states_np.keys():
# #     print(key)

# # print("-- key of states --")
# # print(states_np[key].keys())

# ht = [states_np[key]["beliefs"] for key in states_np.keys()]
# ht = np.vstack(ht)
# pca_ht = get_pca_model(ht)

# st_q = [states_np[key]["posterior_means"] for key in states_np.keys()]
# st_q = np.vstack(st_q)
# pca_st_q = get_pca_model(st_q)

# colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]

# fig = plt.figure(figsize=(6, 6))
# # ax = fig.add_subplot(111, projection="3d")
# ax = fig.add_subplot(111)

# for key in states_np.keys():
#     num = int(key.split("/")[-2])
#     color = colors[num]
    
#     ht = states_np[key]["beliefs"]
#     shape = ht.shape[-1]
#     feat = tensor2np(ht).reshape(-1, shape)
#     feat_pca = pca_ht.transform(feat)
#     x, y, z = get_xyz(feat_pca)
#     ax.plot(x,y,z, alpha=0.2, color=color)
#     # ax.scatter(x, y, z, alpha=0.2)
# plt.show()

# fig = plt.figure(figsize=(6, 6))
# # ax = fig.add_subplot(111, projection="3d")
# ax = fig.add_subplot(111)

# for key in states_np.keys():
#     num = int(key.split("/")[-2])
#     color = colors[num]
    
#     st_q = states_np[key]["posterior_means"]
#     shape = st_q.shape[-1]
#     feat = tensor2np(st_q).reshape(-1, shape)
#     feat_pca = pca_st_q.transform(feat)
#     x, y, z = get_xyz(feat_pca)
#     ax.plot(x, y, alpha=0.2, color=color)
#     # ax.scatter(x, y, z, alpha=0.2)
# plt.show()



### reconstruction result

In [20]:
observations_target = model._clip_obs(observations, idx_start=1)
state = model.estimate_state(observations_target, actions[:-1], rewards, nonterminals[:-1])

In [26]:
print(state["posterior_std_devs"][0:3])

tensor([[[1.0451e+01, 1.0000e-01, 3.6013e+00, 9.2809e+01, 2.2434e+02,
          1.8348e+02, 1.0516e+02, 1.0000e-01, 1.7196e+02, 1.4635e+02,
          1.0000e-01, 1.0000e-01, 1.8327e+01, 1.6882e+02, 1.0000e-01,
          1.0000e-01, 1.3697e+02, 1.4408e+02, 1.5743e+02, 1.0000e-01,
          1.8145e+02, 1.0001e-01, 4.3007e+01, 2.1280e+02, 1.0001e-01,
          1.0000e-01, 1.2953e+02, 1.6238e+02, 1.9647e+02, 8.6683e+01,
          1.8160e+02, 1.0000e-01]],

        [[1.0941e+01, 1.0000e-01, 3.4711e+00, 9.4146e+01, 2.2500e+02,
          1.8422e+02, 1.0552e+02, 1.0000e-01, 1.7242e+02, 1.4732e+02,
          1.0000e-01, 1.0000e-01, 1.9881e+01, 1.6837e+02, 1.0000e-01,
          1.0000e-01, 1.3728e+02, 1.4525e+02, 1.5773e+02, 1.0000e-01,
          1.8137e+02, 1.0001e-01, 4.3098e+01, 2.1270e+02, 1.0002e-01,
          1.0000e-01, 1.3041e+02, 1.6274e+02, 1.9653e+02, 8.6102e+01,
          1.8194e+02, 1.0000e-01]],

        [[1.1066e+01, 1.0000e-01, 3.5666e+00, 9.4783e+01, 2.2544e+02,
          1.8397

In [25]:
state.keys()

dict_keys(['beliefs', 'prior_states', 'prior_means', 'prior_std_devs', 'posterior_states', 'posterior_means', 'posterior_std_devs'])

In [27]:
# recon = model.rssm.observation_model(h_t=state["beliefs"], s_t=state["posterior_states"])
recon = model.observation_model(h_t=state["beliefs"][:1], s_t=state["posterior_means"][:1])
recon.keys()

dict_keys(['image_hsr_256'])

In [28]:
print(recon['image_hsr_256']['loc'])

tensor([[[[[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]],

          [[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]],

          [[nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           ...,
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan],
           [nan, nan, nan,  ..., nan, nan, nan]]]]], device='cuda:0')


In [30]:
print(torch.isnan(state["posterior_means"][:1]).any())

tensor(False, device='cuda:0')


In [19]:
# import cv2
# from utils.evaluation.visualize_utils import reverse_image_observation
# from IPython import display
# import mpl_toolkits

# h_graph = 1
# w_graph = 2
# fig = plt.figure(figsize=(w_graph*5,h_graph*5))
# ax1 = fig.add_subplot(h_graph, w_graph, 1)
# ax2 = fig.add_subplot(h_graph, w_graph, 2)


# obs_name1 = "image_bin"

# n_frame = len(recon[obs_name1]["loc"])

# ax1.axis("off")
# ax2.axis("off")

# colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]


# dt = 1
# size = (64,64)
# artists = []
# for t in range(0, n_frame, dt):
#     im = observations_target[obs_name1][t, 0].detach().cpu().numpy()
#     im = reverse_image_observation(im)
#     im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
#     im1 = ax1.imshow(im, cmap="gray")
#     ax1.set_title("Observation")
    
#     im = recon[obs_name1]["loc"][t, 0].detach().cpu().numpy()
#     im = reverse_image_observation(im)
#     im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
#     im2 = ax2.imshow(im, cmap="gray")
#     ax2.set_title("Reconstruction")
    
#     artists.append([im1,im2])

# # 4. アニメーション化
# anim = ArtistAnimation(fig, tqdm(artists), interval=100*dt)

# if use_validation_data:
#     folder_name = "figs/validation"
# else:
#     folder_name = "figs/train"

# os.makedirs(folder_name, exist_ok=True)
# save_file_name = "{}/reconstruction_ep{}.mp4".format(folder_name, epi_idx)
# anim.save(save_file_name, writer='ffmpeg')
# plt.show()

# print("fin")

In [20]:
import matplotlib.animation as animation

import cv2
from torch import zeros
from utils.evaluation.visualize_utils import reverse_image_observation
from IPython import display
import mpl_toolkits


obs_name1 = "image_hsr_256"

n_frame = len(recon[obs_name1]["loc"])

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]

h_graph = 1
w_graph = 2

dt = 1
size = (64,64)


def plot_1frame(t=0):
    fig = plt.figure(figsize=(w_graph*5,h_graph*5))
    ax1 = fig.add_subplot(h_graph, w_graph, 1)
    ax2 = fig.add_subplot(h_graph, w_graph, 2)
    
    ax1.axis("off")
    ax2.axis("off")

        
    im = observations_target[obs_name1][t, 0][[2,1,0]].detach().cpu().numpy()
    im = reverse_image_observation(im)
    im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
    im1 = ax1.imshow(im)
    ax1.set_title("Observation")
    
    im = recon[obs_name1]["loc"][t, 0][[2,1,0]].detach().cpu().numpy()
    im = reverse_image_observation(im)
    im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
    im2 = ax2.imshow(im)
    ax2.set_title("Reconstruction")
    
    

In [21]:
from ipywidgets import interact


interact(plot_1frame, t=(0,n_frame-1));

interactive(children=(IntSlider(value=0, description='t', max=2966), Output()), _dom_classes=('widget-interact…

# Imagination

In [22]:
t_imag_start = 50

h_t = [state["beliefs"][t_imag_start]]
# s_t = [state["posterior_states"][t_imag_start]]
s_t = [state["posterior_means"][t_imag_start]]

t_max = len(actions)
horizon_imagination = t_max-t_imag_start
for t in range(horizon_imagination):
    belief, _, prior_mean, _ = model.transition_model(s_t[t], actions[t_imag_start+t].unsqueeze(dim=0), h_t[t], det=True)
    h_t.append(belief.squeeze(dim=0))
    # st.append(prior_state.squeeze(dim=0))
    s_t.append(prior_mean.squeeze(dim=0))

In [23]:
h_t = torch.stack(h_t)
s_t = torch.stack(s_t)


In [24]:
recon_imag = model.observation_model(h_t=h_t, s_t=s_t)
# recon_imag["image_vertical_high_resolution"]["loc"].shape

In [25]:
# import cv2
# from utils.evaluation.visualize_utils import reverse_image_observation
# from IPython import display
# import mpl_toolkits

# h_graph = 1
# w_graph = 2
# fig = plt.figure(figsize=(w_graph*5,h_graph*5))
# ax1 = fig.add_subplot(h_graph, w_graph, 1)
# ax2 = fig.add_subplot(h_graph, w_graph, 2)


# obs_name1 = "image_hsr_256"

# n_frame = horizon_imagination

# ax1.axis("off")
# ax2.axis("off")

# colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]


# dt = 1
# size = (64,64)
# artists = []
# for t in range(0, n_frame, dt):
#     t_obs = t_imag_start+t
    
#     if t_obs >= len(observations[obs_name1]):
#         t_obs = len(observations[obs_name1])-1
    
#     im = observations[obs_name1][t_obs, 0].detach().cpu().numpy()
#     im = reverse_image_observation(im)
#     im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
#     im1 = ax1.imshow(im, cmap="gray")
#     ax1.set_title("Observation")
    
#     im = recon_imag[obs_name1]["loc"][t, 0].detach().cpu().numpy()
#     im = reverse_image_observation(im)
#     im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
#     im2 = ax2.imshow(im, cmap="gray")
#     ax2.set_title("Imagination")
    
#     artists.append([im1,im2])


# # 4. アニメーション化
# anim = ArtistAnimation(fig, tqdm(artists), interval=100*dt)

# if use_validation_data:
#     folder_name = "figs/validation"
# else:
#     folder_name = "figs/train"

# os.makedirs(folder_name, exist_ok=True)
# save_file_name = "{}/imagination_ep{}.mp4".format(folder_name, epi_idx)
# anim.save(save_file_name, writer='ffmpeg')
# plt.show()

# print("fin")

In [26]:
import matplotlib.animation as animation

import cv2
from torch import zeros
from utils.evaluation.visualize_utils import reverse_image_observation
from IPython import display
import mpl_toolkits


obs_name1 = "image_hsr_256"

colors = ["tab:blue", "tab:orange", "tab:green", "tab:red", "tab:purple", "tab:brown", "tab:pink", "tab:gray", "tab:olive", "tab:cyan"]

h_graph = 1
w_graph = 2

dt = 1
size = (64,64)


def plot_1frame(t=0):
    fig = plt.figure(figsize=(w_graph*5,h_graph*5))
    ax1 = fig.add_subplot(h_graph, w_graph, 1)
    ax2 = fig.add_subplot(h_graph, w_graph, 2)
    
    ax1.axis("off")
    ax2.axis("off")
    t_obs = t_imag_start+t
    
    if t_obs >= len(observations[obs_name1]):
        t_obs = len(observations[obs_name1])-1
    
    im = observations[obs_name1][t_obs, 0][[2,1,0]].detach().cpu().numpy()
    im = reverse_image_observation(im)
    im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
    im1 = ax1.imshow(im)
    ax1.set_title("Observation")
    
    im = recon_imag[obs_name1]["loc"][t, 0][[2,1,0]].detach().cpu().numpy()
    im = reverse_image_observation(im)
    im = cv2.resize(im, size, interpolation=cv2.INTER_LINEAR)
    im2 = ax2.imshow(im)
    ax2.set_title("Imagination")
    
    

In [27]:
from ipywidgets import interact

n_frame = horizon_imagination
interact(plot_1frame, t=(0,n_frame-1));

interactive(children=(IntSlider(value=0, description='t', max=2917), Output()), _dom_classes=('widget-interact…